# How to connect to OpenAI service via PromptSail and OpenAI Python SDK

First import the OpenAI Python SDK and load your API key from the environment.


In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint

config = dotenv_values(".env")

openai_key = config["OPENAI_API_KEY"]
openai_org_id = config["OPENAI_ORG_ID"]
print(
    f"OpenAI api key={openai_key[0:3]}...{openai_key[-3:]}"
)

OpenAI api key=sk-...Uzy


Lets see what gpt models are available for the API key we loaded.

In [3]:
client_org = OpenAI(
    organization=openai_org_id,
    api_key=openai_key,
)
models = client_org.models.list().data
models = [m for m in models if "gpt" in m.id.lower()]

# sort models by model id
models = sorted(models, key=lambda x: x.id)

# print models which contains gpt in the name
pprint(models)

[Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0301', created=1677649963, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-1106', created=1698959748, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='gpt-3.5-turbo-16k-0613', created=1685474247, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'),
 Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
 Model(id='gpt-4-0314', created=1687882410, object='model', owned_by='openai'),
 Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'),
 Model(id='gpt-4-1106-preview', created=1

Test the direct connection to OpenAI service via OpenAI Python SDK.

In [4]:
oai_client = OpenAI(api_key=openai_key)


response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
        },
        {
            "role": "user",
            "content": "Compose a poem that explains the concept of recursion in programming.",
        },
    ],
)

pprint(response.choices[0].message.content)


('Amidst the realm of code and lines,\n'
 'Recursion whispers, its voice entwines.\n'
 'A concept abstract, yet deeply profound,\n'
 'Where algorithms dance, a symphony resound.\n'
 '\n'
 'In a world of loops, where patterns twist,\n'
 'Recursion emerges, persistent and persist.\n'
 'A self-referential enchantment it weaves,\n'
 'Unraveling mysteries, like autumnal leaves.\n'
 '\n'
 'Like a Russian doll, nesting with grace,\n'
 'Recursion explores, in a mystical chase.\n'
 'A function calls itself, again and again,\n'
 'Unraveling complexity, chasing the zen.\n'
 '\n'
 'With elegance and grace, it navigates,\n'
 'Onwards it journeys, as it creates,\n'
 'Unraveling puzzles, one layer at a time,\n'
 'Looping within, in a parallel rhyme.\n'
 '\n'
 'Each iteration brings forth new birth,\n'
 'A fractal unraveled, expanding its girth.\n'
 'In a recursive embrace, it finds its home,\n'
 'Revealing wonders, where answers roam.\n'
 '\n'
 'Iteration within iteration, it soars high,\n'
 'An infi

Testing the streaming API

In [16]:
oai_client = OpenAI(api_key=openai_key)


response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    stream=True,
    messages=[
        {
            "role": "system",
            "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
        },
        {
            "role": "user",
            "content": "Compose a haiku that explains the concept of recursion in programming.",
        },
    ],
)

for chunk in response:
    print(chunk.choices[0].delta.content)




Nested
 loops
,
 endless


Function
 calls
 itself
,
 yet


One
 task
 becomes
 all
None


## Create a request to the OpenAI via promptsail proxy

Run the docker and go to PromptSail UI http://promptsail.local/


At the application start we created two test projects with OpenAI API mappings. We will use project1 for this example.

* [project1](http://promptsail.local/ui/project/project1) -> https://api.openai.com/v1

You can create your own project if you want.



In [6]:
ps_api_base = "http://promptsail.local:8000/project1/"

# adress with tags 
# ps_api_base = "http://promptsail.local:8000/project1/?tags=tag1,tag2&target_path="


ps_client = OpenAI(base_url=ps_api_base, api_key=openai_key, max_retries=0)


response = ps_client.chat.completions.create(
    model="gpt-3.5-turbo",
    timeout=1000,
    messages=[
        {
            "role": "system",
            "content": "Yoda assistant you are, skilled in explaining complex life and phisopical matters.",
        },
        {
            "role": "user",
            "content": "What number 42 means",
        },
    ],
)

pprint(response.choices[0].message.content)

NotFoundError: Error code: 404 - {'error': {'message': 'Invalid URL (POST /v1//chat/completions)', 'type': 'invalid_request_error', 'param': None, 'code': None}}